# Imports

In [51]:
import numpy as np # Pour les math
#import matplotlib.pyplot as plt # Pour les graphiques
from scipy.optimize import linprog # Notre solveur

# Données et Utils

In [52]:
temperatures_exterieurs = np.load("Temperatures-Montreal.npy")
print(temperatures_exterieurs[0])

-0.8299999999999841


In [53]:
def is_night(idx): # run in O(n) instead of O(1) because of flemme normally should be done with mod and divs
    # idx is 15
    # each day is 96 interval
    #each hour is 4 interval
    #so from 0 to 24 is night ...
    #28 to 84 is day ...
    #88 to 96 is night...
    real_idx = 0
    for i in range(0,len(temperatures_exterieurs)):
        if real_idx > 96: #principe du mod 96 mais faut faire le reste des calculs et c'est pas le plus important
            real_idx = 0
        if(i == idx):
            if(real_idx >= 0 and real_idx <= 24):
                return True
            elif(real_idx >= 28 and real_idx <= 84):
                return False
            elif(real_idx >= 28 and real_idx <= 96):
                return True
        real_idx += 1

# Résolution du système
## Variables de décison
Soit $x_i l'énergie utiliser en mode normale par la pompe à chaleur à l'intervalle de temps i. \
Soit $y_i l'énergie utiliser en mode reverse par la pompe à chaleur à l'intervalle de temps i. 

## Objectif (FAUX j'ai mal compris l'énoncé ....)
Minimiser le coût total de l'électricité consommée par la pompe à chaleur. \

Fonction objective: \
$min(x_i + y_i) * c_i$

où c_i est le coût de l'électricité à l'intervalle de temps i. \
$c_i = 0.18$ si c'est la nuit \
ou \
$c_i = 0.26$ si c'est le jour


## Contraintes
$x_i \in \{0, 1\}$ \
$y_i \in \{0, 1\}$ \
$x_i \ge (19-t_i)/COP_{xi}$ \
$x_i \le (21-t_i)/COP_{xi}$ \
$y_i \ge -(21-t_i)/3.2$ \
$y_i \le -(19-t_i)/3.2$ \
$t_{{in}_{i+1}} - t_{{in_final}_i} = -0.01 * (t_{{in_final}_i} - t_{{ex}_i})$ \
$COP_{xi} = 3 + 10 * abs(tanh(t_{{ex}_i}/100)) * tanh(t_{{ex}_i}/100)$ \
où $t_i$ est la température intérieur initiale à l'intervalle de temps i, $COP_{xi}$ le coefficent de performance de la pompe à chaleur en mode noramel à l'intervalle de temps i, $t_{{in_final}_i}$ la température à l'intérieur après avoir utilisé (ou non) la pompe à chaleur à l'intervalle de temps i et $t_{{ex}_i}$ est la température extérieur de Montréal à l'interval de temps i.


# Système entier
$A * x = b$


In [54]:
def task_1(i_idx):
    days_offset = 96
    end_idx = i_idx + days_offset*7
    central_value = 20
    #1h = 4 intervale
    #donc 
    
    # On suppose qu'on commence la journée à 0
    #On crée la matrice, il y a 2 inconnues a trouver donc 2 collones
    # a, d
    valeur_energie = []
    valeur_cout = []
    cop_const = 3.2
    T0 = 20 # température de départ
    for i in range(i_idx, end_idx):
        s = is_night(i)
        elec_cost =  0.18 if s == True else 0.26
        cop_tex = 3 + 10 * abs(np.tanh(temperatures_exterieurs[i]/100)) * np.tanh(temperatures_exterieurs[i]/100)
        
        T1 = (-0.01) * (T0 - temperatures_exterieurs[i]) + T0 #température initial suivante

        # x et y compris entre 0 et 1
        xi_bounds = (0, 1)
        yi_bounds = (0, 1)
        # Fonction objectif = c^T x avec c = (elec_cost,elec_cost)
        ci = (elec_cost, elec_cost)
        # Contraintes : ici de la forme A x <= b 
        A = [[-1, 0], [1, 0], [0, -1], [0, 1]]
         
        b = [0, 0, 0, 0]
        if T1 <= 19:
            b = [-(19-T1)/cop_tex, (21-T1)/cop_tex, 0, 0]
        if T1 >= 21:
            b = [0, 0, (T1-21)/3.2, -(T1-19)/3.2]
       


        res = linprog(ci, A_ub=A, b_ub=b, bounds=(xi_bounds, yi_bounds), options={"disp": False})
        valeur_energie.append(res.x)
        valeur_cout.append(res.fun)
        
        T0 = T1 + (res.x[0] * cop_tex) - (res.x[1] * cop_const)
    solution = sum(valeur_cout)
    print(solution) 
    return solution
        
        
        


In [55]:
task_1(13050)

3.2863166543349225


3.2863166543349225